### statically-typed language : java,C, C++
 컴파일시에 에러가 검출된다. 타입구분이 명확한것들을 정적타입 랭귀지 라함.
 
### dynamic-typed language : python, javascript, Ruby... 
 인터프리터 언어라고도한다. 타입에 딱히 신경쓰지않아도 개발이가능하며 개발속도 기본적으로 빠르다.
 
IDE에서 statical language 는 코드 suggestion이 잘 적용되지만, python같은 dynamic language 는 지원이 상대적으로 부족하다.

In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import attention
from collections import deque
import pickle
import pyreader
import numpy as np

In [2]:
data= 'data_samples/'
data_path='data_samples/mapping.map'
pattern = 'preprocess.part*'
hidden_size = 10
seq_length = 1
batch_size = 100
num_samples = 0
with open(data_path, "rb") as f:
        word_to_id = pickle.load(f)
vocab_size = len(word_to_id)
num_samples = 10
attention_num=5
max_attention=3
lambda_type='state'
keep_prob=0.9,
num_layer=1
max_grad_norm=3
learning_rate = 0.01
files = [y for x in os.walk(data) for y in iglob(os.path.join(x[0], pattern))]

### 전처리를 다음과같이 한다.

#### 변수명:번호 인데 아래와 같이 나온다.

In [3]:
current_file = deque(files).popleft()
with open(current_file, 'rb') as f:
    current_data = pickle.load(f)

In [13]:
current_data[1].inputs.shape

(1, 100)

In [9]:
current_data[1].actual_lengths

[99]

### 예를들어 파일한개를 돌려보면..

In [7]:
inv_map = {v: k for k, v in word_to_id.items()} # 아이디값:변수명 으로 reverse시킴.

for i in range(len(current_data)):
    inputs_raw = [inv_map[k] for k in current_data[i].inputs[0]]
#     inputs_raw = [inv_map[k] for k in current_data[i].identifier_usage[0]]
    inputs_raw= ' '.join(inputs_raw)
    targets_raw = [inv_map[k] for k in current_data[i].targets[0]]
    targets_raws = ' '.join(targets_raw)
print(word_to_id['class'],inv_map[56])
print(inputs_raw)
print('--------------------------------------------------------------------------')
print(targets_raws)

56 class
class §OOV§ ( object ) : 
 §<indent>§ §OOV§ 
 
 def __init__ ( self , §OOV§ , §OOV§ ) : 
 §<indent>§ pass 
 
 §<dedent>§ def __repr__ ( self ) : 
 §<indent>§ §OOV§ 
 if ( self . type in [ §OOV§ , §OOV§ ] ) : 
 §<indent>§ §OOV§ = ( §OOV§ % ( self . type , self . §OOV§ , ord ( self . §OOV§ ) , self . §OOV§ , self . §OOV§ , self . §OOV§ , self . §OOV§ , self . §OOV§ ) ) 
 §<dedent>§ elif ( self .
--------------------------------------------------------------------------
§OOV§ ( object ) : 
 §<indent>§ §OOV§ 
 
 def __init__ ( self , §OOV§ , §OOV§ ) : 
 §<indent>§ pass 
 
 §<dedent>§ def __repr__ ( self ) : 
 §<indent>§ §OOV§ 
 if ( self . type in [ §OOV§ , §OOV§ ] ) : 
 §<indent>§ §OOV§ = ( §OOV§ % ( self . type , self . §OOV§ , ord ( self . §OOV§ ) , self . §OOV§ , self . §OOV§ , self . §OOV§ , self . §OOV§ , self . §OOV§ ) ) 
 §<dedent>§ elif ( self . type


### 위와같이 normalise + indent와 패딩, out of vocabulary의 값들이  나온다.

### 실제로 집어넣을 값들은 숫자로 feeding한다.

In [5]:
current_data[i].inputs[0]

array([ 56,   1,   3, 210,   4,   8,   2,   9,   1,   2,   2,  17,  65,
         3,  11,   6,   1,   6,   1,   4,   8,   2,   9,  82,   2,   2,
        10,  17, 245,   3,  11,   4,   8,   2,   9,   1,   2,  16,   3,
        11,   5,  69,  20,  13,   1,   6,   1,  14,   4,   8,   2,   9,
         1,   7,   3,   1,  23,   3,  11,   5,  69,   6,  11,   5,   1,
         6, 627,   3,  11,   5,   1,   4,   6,  11,   5,   1,   6,  11,
         5,   1,   6,  11,   5,   1,   6,  11,   5,   1,   6,  11,   5,
         1,   4,   4,   2,  10,  52,   3,  11,   5], dtype=int64)

### 위와같이 넣게되어있다.

In [6]:
with tf.Graph().as_default(), tf.Session() as session:
    masks_ = tf.placeholder(tf.bool, [seq_length, batch_size, attention_num], name="masks")
    input_data_ = tf.placeholder(tf.int32, [seq_length, batch_size], name="inputs")
    targets_ = tf.placeholder(tf.float32, [seq_length, batch_size], name="targets")
    
    a = attention.AttentionModel(input_data=input_data_,
                                 targets=targets_,
                                 masks=masks_,
                                 is_training=True,
                                 attention_num= attention_num,
                                 batch_size=batch_size,
                                 hidden_size=hidden_size,
                                 num_samples=num_samples,
                                 seq_length=seq_length,
                                 vocab_size=vocab_size,
                                 lambda_type=lambda_type,
                                
                                 max_attention=max_attention)
    
    session.run(tf.global_variables_initializer())
    loss = a.loss
    minimize = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    
    for i, seq_batch in enumerate(current_data):
        actual_lengths = seq_batch.actual_lengths
        identifier_usage = seq_batch.identifier_usage
        inputs = seq_batch.inputs
        masks = tf.transpose(seq_batch.masks,[0,2,1]).eval()
        targets = seq_batch.targets
    
        feed_dict = {input_data_:inputs, targets_:targets, masks_:masks}
        loss_, _ = session.run([loss, minimize], feed_dict= feed_dict)
        print('loss', loss_[0])

Constructing Attention Cell
Constructing Basic Model
Constructing Attention Model
loss 20.108
loss 12.3831
loss 7.09279
loss 5.86391
loss 5.97036
loss 1.35504
loss 5.34213
loss 6.29
loss 2.34693
loss 5.20732
loss 2.16649
loss 6.44243
loss 5.82552
loss 2.74365
loss 6.66154
loss 6.87446
loss 0.990804
loss 6.1901
loss 6.96051
loss 4.27856
loss 6.58593
loss 6.75524
loss 4.40741
loss 2.02905
loss 0.274461
loss 0.262891
loss 0.618306
loss 5.97895
loss 4.50095
loss 4.00063
loss 5.55282
loss 6.16202
loss 5.74918
loss 1.19708
loss 4.90465
loss 4.73007
loss 4.59521
loss 4.28174
loss 5.0371
loss 2.93205
loss 1.96934
loss 4.57889
loss 5.61728
loss 4.12503
loss 3.84898
loss 1.36641
loss 3.55735
loss 4.0284
loss 4.73237
loss 4.80192
loss 4.21766
loss 4.01107
loss 4.68324
loss 4.87979
loss 4.87639
loss 3.86066
loss 5.09969
loss 3.83171
loss 4.3907
loss 2.89733
loss 4.22103
loss 4.34626
loss 4.65182
loss 4.37999
loss 3.09249
loss 3.40458
loss 3.09618
loss 2.22543
loss 5.25855
loss 4.88266
loss 3.31036

ValueError: Cannot feed value of shape (2, 100) for Tensor 'inputs:0', which has shape '(1, 100)'